In [124]:
import pandas as pd
import json
import numpy as np

In [107]:
def open_jsonl(filename):
    sentences = []
    with open(filename,  encoding="utf8") as json_file:
        json_list = list(json_file)

    for json_str in json_list[:1000]:
        result = json.loads(json_str)
        sentences.append(result)
    
    return pd.DataFrame(sentences)

In [108]:
train = open_jsonl('data/train.jsonl')
test = open_jsonl('data/test.jsonl')

In [109]:
X = train['text']
y = train['lang']

split data

In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [111]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_train_counts = cv.fit_transform(X_train)

In [112]:
pd.DataFrame([cv.vocabulary_])

,здесь,formula_35,гравитационная,постоянная,formula_36,шварцшильдовский,радиус,упомянутого,тела,гравитационное,...,ознаменован,присуждением,theodor,bastard,премий,горгулья,russian,music,awards,этно
0,6617,791,4732,14393,792,21620,16003,20325,19378,4733,...,11818,15234,1129,649,14865,4619,1055,954,643,22075


In [113]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [114]:
from sklearn.naive_bayes import MultinomialNB

In [115]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [121]:
test_sentences = X_test
X_new_counts = cv.transform(test_sentences)
X_new_tfidf = tfidf.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)


ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru
ru


Testing

In [125]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted, target_names=np.unique(y_train)))

              precision    recall  f1-score   support

          ru       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



to do:
- check with bigger dataset